In [1]:
# importacao das bibliotecas utilizadas
import numpy as np
import cv2
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importacao das imagens
foto1a = plt.imread('imgs/foto1A.jpg')
foto1b = plt.imread('imgs/foto1B.jpg')
foto2a = plt.imread('imgs/foto2A.jpg')
foto2b = plt.imread('imgs/foto2B.jpg')

### (1)

In [3]:
foto1a_g = cv2.cvtColor(foto1a, cv2.COLOR_BGR2GRAY)
foto1b_g = cv2.cvtColor(foto1b, cv2.COLOR_BGR2GRAY)
foto2a_g = cv2.cvtColor(foto2a, cv2.COLOR_BGR2GRAY)
foto2b_g = cv2.cvtColor(foto2b, cv2.COLOR_BGR2GRAY)

### (2)

In [21]:
sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
star = cv2.xfeatures2d.StarDetector_create()
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

In [5]:
# SIFT
sift_kp_foto1a_g, sift_f_foto1a_g = sift.detectAndCompute(foto1a_g,None)
sift_kp_foto1b_g, sift_f_foto1b_g = sift.detectAndCompute(foto1b_g,None)
sift_kp_foto2a_g, sift_f_foto2a_g = sift.detectAndCompute(foto2a_g,None)
sift_kp_foto2b_g, sift_f_foto2b_g = sift.detectAndCompute(foto2b_g,None)

In [6]:
sift_kp_foto1a_g = np.float32([kp.pt for kp in sift_kp_foto1a_g])
sift_kp_foto1b_g = np.float32([kp.pt for kp in sift_kp_foto1b_g])
sift_kp_foto2a_g = np.float32([kp.pt for kp in sift_kp_foto2a_g])
sift_kp_foto2b_g = np.float32([kp.pt for kp in sift_kp_foto2b_g])

In [7]:
# SURF
surf_kp_foto1a_g, surf_f_foto1a_g = surf.detectAndCompute(foto1a_g,None)
surf_kp_foto1b_g, surf_f_foto1b_g = surf.detectAndCompute(foto1b_g,None)
surf_kp_foto2a_g, surf_f_foto2a_g = surf.detectAndCompute(foto2a_g,None)
surf_kp_foto2b_g, surf_f_foto2b_g = surf.detectAndCompute(foto2b_g,None)

In [8]:
surf_kp_foto1a_g = np.float32([kp.pt for kp in surf_kp_foto1a_g])
surf_kp_foto1b_g = np.float32([kp.pt for kp in surf_kp_foto1b_g])
surf_kp_foto2a_g = np.float32([kp.pt for kp in surf_kp_foto2a_g])
surf_kp_foto2b_g = np.float32([kp.pt for kp in surf_kp_foto2b_g])

In [22]:
# BRIEF
kp_foto1a_g = star.detect(foto1a_g,None)
kp_foto1b_g = star.detect(foto1b_g,None)
kp_foto2a_g = star.detect(foto2a_g,None)
kp_foto2b_g = star.detect(foto2b_g,None)

In [23]:
a, b = brief.compute(foto1a_g,kp_foto1a_g)

### (3) e (4)

In [9]:
matcher = cv2.DescriptorMatcher_create("BruteForce")
ratio = 0.8

In [ ]:
rawMatches_foto1 = matcher.knnMatch(sift_f_foto1a_g, sift_f_foto1b_g, 2)

In [ ]:
matches_foto1 = []
for m in rawMatches_foto1:
    if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        matches_foto1.append((m[0].trainIdx, m[0].queryIdx))

### (5)

In [ ]:
def calculate_homography(matches, kpsA, kpsB, thr = 4.0):
    if len(matches) > 4:
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
        
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, thr)
        
        return (H, status)
    return None

In [ ]:
h_foto1 = calculate_homography(matches_foto1, sift_kp_foto1a_g, sift_kp_foto1b_g)

### (6)

In [ ]:
persp_foto1 = cv2.warpPerspective(foto1a_g, h_foto1[0], (foto1a_g.shape[1] + foto1b_g.shape[1], foto1a_g.shape[0]))

### (7)

In [ ]:
persp_foto1[0:foto1b_g.shape[0], 0:foto1b_g.shape[1]] = foto1b_g

### (8)

In [ ]:
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
            
    return vis

In [ ]:
vis_foto1 = drawMatches(foto1a, foto1b, sift_kp_foto1a_g, sift_kp_foto1b_g, matches_foto1, h_foto1[1])

In [ ]:
plt.imshow(vis_foto1,'gray')